In [1]:
import nidaqmx
from nidaqmx.constants import *
import numpy as np
from nidaqmx import stream_readers
import time


In [2]:


params = {
    'sampling_rate' : 100000,
    'acq_time' : 100, ###in ms
    'number_samples' : 100000 * 100 * 0.001,
    'clock_port' : "Dev1/port0",
    'count_dev' : "Dev1/",
    'count_port' : "ctr0",
    'edge_type' : Edge.RISING,
    'trig_edge_type' : Edge.RISING,
    'trig_type' : TriggerType.DIGITAL_EDGE,
    'pfi_port' : "PFI0"
}

In [3]:
def init_clock(port, sampling_rate):
    sample_clock = nidaqmx.Task()
    sample_clock.di_channels.add_di_chan(port)
    sample_clock.timing.cfg_samp_clk_timing(rate= sampling_rate, sample_mode = AcquisitionType.CONTINUOUS)
    sample_clock.control(TaskMode.TASK_COMMIT)
    return sample_clock


def init_read(dev, port, edge_type, trig_edge_type, trig_type, pfi_port, sampling_rate):
    read_task = nidaqmx.Task()
    read_task.ci_channels.add_ci_count_edges_chan(
                                dev + port,
                                edge=edge_type,
                                initial_count=0,
                                count_direction=CountDirection.COUNT_UP)
    read_task.ci_channels.all.ci_count_edges_term = "/"+ dev + pfi_port
    
    read_task.timing.cfg_samp_clk_timing(sampling_rate, source="/" + dev + "di/SampleClock",
        active_edge=Edge.RISING, sample_mode=AcquisitionType.CONTINUOUS)

    # read_task.in_stream.input_buf_size = 120000000
    
    read_task.triggers.arm_start_trigger.trig_type = trig_type
    read_task.triggers.arm_start_trigger.dig_edge_edge = trig_edge_type
    read_task.triggers.arm_start_trigger.dig_edge_src = "/"+ dev+"di/SampleClock"

    return read_task

def run_counter(read_task, clock_task, number_samples, acq_time):
    number_samples = int(number_samples)
    reader = stream_readers.CounterReader(read_task.in_stream)
    try:
        clock_task.start()
        read_task.start()
    except: print('')
    
    data_array = np.empty(number_samples, dtype = np.uint32)
    
    reader.read_many_sample_uint32(data_array,
    number_of_samples_per_channel=number_samples, timeout=0.001*acq_time)
    return data_array

def stop_counter(read_task, clock_task):
    clock_task.stop()
    
    read_task.stop()
    
def close_counter(read_task, clock_task):
    clock_task.close()
    read_task.close()
    
def init_all(sampling_rate, acq_time, number_samples, clock_port, count_dev, count_port, edge_type, trig_edge_type, trig_type, pfi_port, close=False):
    clock_task = init_clock(clock_port, sampling_rate)

    read_task = init_read(count_dev, count_port, edge_type, trig_edge_type, trig_type, pfi_port, sampling_rate)

    return clock_task, read_task

def get_countrate(sampling_rate, acq_time, number_samples, clock_port, count_dev, count_port, edge_type, trig_edge_type, trig_type, pfi_port, close=False):
    clock_task = init_clock(clock_port, sampling_rate)

    read_task = init_read(count_dev, count_port, edge_type, trig_edge_type, trig_type, pfi_port, sampling_rate)

    counter = run_counter(read_task, clock_task, number_samples, acq_time)

    stop_counter(read_task, clock_task)

    time = acq_time * 0.001
    # print(counter)
    countrate = counter[-1] /time
    print(countrate)
    if close==True:
        close_counter(read_task, clock_task)

    return countrate

In [ ]:
clock_task, read_task = init_all(**params)

while True:
    counter = run_counter(read_task, clock_task, params['number_samples'], params['acq_time'])

    stop_counter(read_task, clock_task)

    time = params['acq_time'] * 0.001
    # print(counter)
    countrate = counter[-1] /time
    print(countrate)


0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
